In [3]:
#using pandas to clean up the what we scraped from the bbc website

#import pandas
import pandas as pd

#read in the csv 
df = pd.read_csv('bbcmovies.csv')

In [4]:
df


,Unnamed: 0.1,Unnamed: 0,movie_name,movie_rank,movie_year,movie_dir,critic_name,critic_org,critic_cn
0,0,0,Mulholland Drive,1,2001,David Lynch,Simon Abrams,Freelance film critic,US
1,1,1,In the Mood for Love,2,2000,Wong Kar-wai,Simon Abrams,Freelance film critic,US
2,2,2,The Tree of Life,3,2011,Terrence Malick,Simon Abrams,Freelance film critic,US
3,3,3,Yi Yi: A One and a Two,4,2000,Edward Yang,Simon Abrams,Freelance film critic,US
4,4,4,Goodbye to Language,5,2014,Jean-Luc Godard,Simon Abrams,Freelance film critic,US
...,...,...,...,...,...,...,...,...,...
1765,1765,1765,The Lives of Others,6,2006,Florian Henckel von Donnersmarck,Raymond Zhou,China Daily,China
1766,1766,1766,Still Life,7,2006,Jia Zhangke,Raymond Zhou,China Daily,China
1767,1767,1767,Birdman,8,2014,Alejandro González Iñárritu,Raymond Zhou,China Daily,China
1768,1768,1768,Infernal Affairs,9,2002,Andrew Lau and Alan Mak,Raymond Zhou,China Daily,China


In [5]:
#drop the index columns that are unnamed
df = df.drop('Unnamed: 0.1',axis=1)

In [6]:
df = df.drop('Unnamed: 0',axis=1)

In [7]:
#get a list of all critic countries so you can manually pick out asian countries

df.critic_cn.value_counts()

US              820
UK              180
Canada           50
India            50
Germany          50
France           50
Cuba             50
Italy            40
Israel           40
Australia        40
Colombia         40
UAE              30
Lebanon          30
Mexico           20
Austria          20
Singapore        20
Chile            20
Argentina        20
South Korea      20
Turkey           20
Qatar            10
Bangladesh       10
Taiwan           10
Switzerland      10
Hong Kong        10
Namibia          10
Indonesia        10
Belgium          10
Kazakhstan       10
South Africa     10
Japan            10
Egypt            10
Philippines      10
Senegal          10
Brazil           10
China            10
Name: critic_cn, dtype: int64

In [12]:
#assign all asian countries to a variable
filter_values = ['India','Singapore','South Korea','Taiwan','Hong Kong','Indonesia','Kazakhstan','Japan','Phillipines','China']

#make a df of asian countries
dfasia = df[df.critic_cn.isin(filter_values)]
#df[df.critic_cn == 'China']

#save the df to a csv
dfasia.to_csv("asiacriticsmovies.csv")
dfasia.value_counts()

movie_name                movie_rank  movie_year  movie_dir                        critic_name      critic_org                         critic_cn
 35 Shots of Rum          9           2008        Claire Denis                     Maggie Lee       Variety                            Hong Kong    1
 Pan's Labyrinth          10          2006        Guillermo Del Toro               Laya Maheshwari  Freelance film critic              India        1
 Shrek                    10          2001        Andrew Adamson and Vicky Jenson  Alaka Sahani     The Indian Express                 India        1
 Son of Saul              6           2015        László Nemes                     Utpal Borpujari  Freelance film critic              India        1
 Spirited Away            7           2001        Hayao Miyazaki                   Maggie Lee       Variety                            Hong Kong    1
                                                                                                         

In [9]:
#here i'm taking out directors where thre was more than one director. i only did this to help troubleshoot my IMDB scrape. Checking df asia value counts shows there are 12 rows where there are two or more directors. Will add those in later. 
twomore_dir = df.movie_dir.str.contains('and')

In [10]:
twomore_dir.value_counts()

False    1627
True      143
Name: movie_dir, dtype: int64

In [27]:
#make a df of only one director per movie
dfnoands = df[df['movie_dir'].str.contains('and')==False ]

In [28]:
#save it to csv
dfnoands.to_csv('bbcmovies1d.csv')

In [37]:
#get asian critics from this 
 dfasianoands = dfnoands[dfnoands.critic_cn.isin(filter_values)]

In [38]:
dfasianoands

,movie_name,movie_rank,movie_year,movie_dir,critic_name,critic_org,critic_cn
210,Amour,1,2012,Michael Haneke,Utpal Borpujari,Freelance film critic,India
211,A Separation,2,2011,Asghar Farhadi,Utpal Borpujari,Freelance film critic,India
212,In the Mood for Love,3,2000,Wong Kar-wai,Utpal Borpujari,Freelance film critic,India
213,A Prophet,4,2009,Jacques Audiard,Utpal Borpujari,Freelance film critic,India
214,Monsoon Wedding,5,2001,Mira Nair,Utpal Borpujari,Freelance film critic,India
...,...,...,...,...,...,...,...
1762,The Sun Also Rises,3,2007,Jiang Wen,Raymond Zhou,China Daily,China
1763,A Separation,4,2011,Asghar Farhadi,Raymond Zhou,China Daily,China
1764,"Lust, Caution",5,2007,Ang Lee,Raymond Zhou,China Daily,China
1765,The Lives of Others,6,2006,Florian Henckel von Donnersmarck,Raymond Zhou,China Daily,China


In [39]:
#formatting it for mapping
dfasiamoviecount = dfasianoands.groupby('critic_cn')['movie_name'].value_counts().reset_index(name='count')

In [48]:
#adding count of vote per critic
import numpy as np
dfasiamoviecount["string"] = dfasiamoviecount["movie_name"] + ": " + dfasiamoviecount["count"].astype(str) + np.where(dfasiamoviecount["count"]>1, ' votes', ' vote')


In [49]:
dfasiamoviecount

,critic_cn,movie_name,count,string
0,China,A Separation,1,A Separation : 1 vote
1,China,"Lust, Caution",1,"Lust, Caution : 1 vote"
2,China,"Spring, Summer, Fall, Winter…and Spring",1,"Spring, Summer, Fall, Winter…and Spring : 1 vote"
3,China,Still Life,1,Still Life : 1 vote
4,China,The Hours,1,The Hours : 1 vote
...,...,...,...,...
125,Taiwan,Mulholland Drive,1,Mulholland Drive : 1 vote
126,Taiwan,Spirited Away,1,Spirited Away : 1 vote
127,Taiwan,The Assassin,1,The Assassin : 1 vote
128,Taiwan,Tropical Malady,1,Tropical Malady : 1 vote


In [50]:
output = dfasiamoviecount.groupby('critic_cn')['string'].apply(lambda x: "<div id='movie'><h1><b>Top Movies</b></h1><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.article')
output

,critic_cn,properties.article
0,China,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
1,Hong Kong,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
2,India,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
3,Indonesia,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
4,Japan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
5,Kazakhstan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
6,Singapore,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
7,South Korea,<div id='movie'><h1><b>Top Movies</b></h1><P> ...
8,Taiwan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...


In [52]:
output.iloc[3]['properties.article']

"<div id='movie'><h1><b>Top Movies</b></h1><P> A Separation : 1 vote</p><p>  Caché : 1 vote</p><p>  Norte, the End of History : 1 vote</p><p>  Nostalgia for the Light : 1 vote</p><p>  Spirited Away : 1 vote</p><p>  The Act of Killing : 1 vote</p><p>  The Gleaners and I : 1 vote</p><p>  The Great Beauty : 1 vote</P></div>"

In [55]:
#getting headline
crits = dfasianoands.groupby('critic_cn')['critic_name'].nunique().reset_index(name='properties.headline')
crits

,critic_cn,properties.headline
0,China,1
1,Hong Kong,1
2,India,5
3,Indonesia,1
4,Japan,1
5,Kazakhstan,1
6,Singapore,2
7,South Korea,2
8,Taiwan,1


In [56]:
#merging articles and headline
output = output.merge(crits, how='left', on='critic_cn')
output

,critic_cn,properties.article,properties.headline
0,China,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1
1,Hong Kong,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1
2,India,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,5
3,Indonesia,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1
4,Japan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1
5,Kazakhstan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1
6,Singapore,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,2
7,South Korea,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,2
8,Taiwan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1


In [57]:
output['properties.headline'] = output['properties.headline'].map(str) + np.where(output["properties.headline"]>1, ' critics', ' critic')

In [58]:
#getting number of critics per country
output

,critic_cn,properties.article,properties.headline
0,China,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic
1,Hong Kong,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic
2,India,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,5 critics
3,Indonesia,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic
4,Japan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic
5,Kazakhstan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic
6,Singapore,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,2 critics
7,South Korea,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,2 critics
8,Taiwan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic


In [59]:
#adding color to country

output['properties.color'] = "#35476E"
output

,critic_cn,properties.article,properties.headline,properties.color
0,China,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic,#35476E
1,Hong Kong,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic,#35476E
2,India,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,5 critics,#35476E
3,Indonesia,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic,#35476E
4,Japan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic,#35476E
5,Kazakhstan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic,#35476E
6,Singapore,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,2 critics,#35476E
7,South Korea,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,2 critics,#35476E
8,Taiwan,<div id='movie'><h1><b>Top Movies</b></h1><P> ...,1 critic,#35476E
